## **1.1 Why one might expect these nine factors to be related to stock returns?**

**1. MKT (market):** The market factor captures the overall compensation investors require for bearing systematic risk. Since most stocks co-move with the market, exposure to aggregate risk drives expected returns. Higher beta stocks tend to earn higher returns as compensation for higher covariance with the market.

**2. SMB (size):** Smaller firms are riskier due to less diversified operations, higher distress risk, and lower liquidity, which leads investors to demand higher expected returns. Behavioral explanations (such as limited attention to small stocks) also support a persistent size premium.

**3. HML (value):** Value stocks (high book-to-market) often face financial distress or lower growth prospects, increasing their risk and required return. Additionally, investors may overreact to past bad performance, making value stocks undervalued and yielding higher subsequent returns.

**4. RMW (profitability):** Firms with robust profitability tend to have more stable cash flows and lower risk, yet markets may underprice these firms due to slow information diffusion. High-profitability firms therefore earn higher risk-adjusted returns as compensation for underlying economic risks or correction of mispricing.

**5. CMA (investments):** Firms that invest conservatively are typically more disciplined and face fewer financing frictions, leading to higher expected returns. Conversely, aggressive investment often signals lower future profitability, so low-investment firms outperform high-investment firms.

**6. UMD (momentum):** Stocks with strong recent performance tend to continue outperforming in the short to medium term due to underreaction, slow learning, or behavioral biases among investors. Momentum is thus linked to return persistence and delayed price adjustment.

**7. ROE (profitability):** High ROE indicates efficient use of equity capital and strong earnings generation, which predicts higher future cash flows. Markets may underreact to persistent profitability, leading high-ROE firms to earn higher future returns.

**8. IA (investments):** Low asset-growth firms tend to outperform because high asset growth is often associated with managerial overinvestment when firm valuations are high. From a risk perspective, firms expanding aggressively are more vulnerable to future cash-flow downturns, justifying lower expected returns.

**9. BAB (betting against beta):** High-beta stocks often become overpriced due to leverage constraints that push constrained investors toward higher-beta assets. As a result, low-beta stocks earn higher risk-adjusted returns, making the BAB factor profitable as it exploits this mispricing or risk-based leverage premium.


**Difference between RMW & ROE:**  
RMW = operating profitability–based factor (pre-financing), portfolio return.  
Definition: (revenues – cost of goods sold – selling/general/admin expenses – interest expense) divided by book equity.  
Fama–French argue that operating profitability is more directly tied to future cashflows, avoiding distortions from leverage and accounting noise.

ROE = accounting profitability (post-financing), firm-level measure used to build a different profitability factor.  
Definition: Net Income / Book Equity.  
Hou–Xue–Zhang argue ROE is a stronger predictor of expected returns due to its relation to economic theory (investment–profitability dynamics).

**Difference between CMA & IA:**  
CMA = asset growth / total investment.  
CMA is a portfolio-based factor return from Fama–French that captures return differences between low- and high-investment firms.

IA = investment / begining of period total assets  
IA is a firm-level accounting ratio measuring investment intensity and used in the q-factor model, not a factor return itself.

## **1.2**
Find the optimal $\theta$ vector (of dimension 9 x 1) for a mean-variance investor with risk aversion of $\gamma$ = 5 if the investor can invest in only these nine factors. Use the entire dataset to estimate the nine factors' mean and covariance of returns

In [34]:
import numpy as np
import pandas as pd

df_data = pd.read_excel('QPM-FactorsData-ForAssignment-04.xlsx', index_col=0)
df_data.index = pd.to_datetime(df_data.index.astype(str), format='%Y%m') + pd.offsets.MonthEnd(0)
print('Çheck if returns have null values\n',df_data.isnull().sum())

def get_theta(returns, risk_aversion):
    """
    Perform mean-variance optimization.
    
    θ* = (1/γ) * Σ^(-1) * μ
    
    Then normalize weights to sum to 1.
    
    Parameters:
    -----------
    returns_df : pd.DataFrame
        DataFrame of asset returns
    risk_aversion : float
        Risk aversion parameter γ
        
    Returns:
    --------
    pd.Series : Normalized portfolio weights
    """
    mu = returns.mean()
    sigma = returns.cov()

    #handle case where sigma is singular (not invertible)
    #this can when computing sigma for volatility-timed returns
    try:
        sigma_inv = np.linalg.inv(sigma)
    except np.linalg.LinAlgError:
        sigma_inv = np.linalg.pinv(sigma)

    # we do not need to annualize mu and sigma here because they both have the same time scaling effect. 
    # In the calculation of theta, the time scaling effect cancels out.
    weights = (1/risk_aversion) * sigma_inv @ mu
    return pd.Series(weights, index=returns.columns)

print('='*60)
gamma = 5    
get_theta(df_data, risk_aversion=gamma).rename('Factor Weights').round(4)

Çheck if returns have null values
 Market    0
SMB       0
HML       0
RMW       0
CMA       0
UMD       0
ROE       0
IA        0
BAB       0
dtype: int64


Market    1.1062
SMB       0.9558
HML      -0.1550
RMW       0.2070
CMA       0.1534
UMD       0.2162
ROE       1.7734
IA        2.7261
BAB       0.7783
Name: Factor Weights, dtype: float64

## **1.3**
Find the Sharpe ratio for each of the nine individual factors and compare it to the Sharpe ratio of the parametric portfolio you have identified in the previous question.

In [35]:
def sharpe_ratio(returns, periods_per_year=12):
    mu = returns.mean() 
    volatility = returns.std()
    sharpe_ratio = (mu / volatility) 
    # return annualized Sharpe Ratios
    return sharpe_ratio * np.sqrt(periods_per_year)

# sharpe ratio of the porfolio identified in Q1.2
theta = get_theta(df_data, risk_aversion=gamma)
mu_portfolio = theta @ df_data.mean()
volatility_portfolio = np.sqrt(theta @ df_data.cov() @ theta)
sharpe_portfolio = mu_portfolio / volatility_portfolio * np.sqrt(12)

sr_factors = sharpe_ratio(df_data).rename('Sharpe Ratio')
sr_factors['PORTFOLIO'] = sharpe_portfolio
sr_factors.round(4)

Market       0.4329
SMB          0.1979
HML          0.2779
RMW          0.4207
CMA          0.4779
UMD          0.5073
ROE          0.6843
IA           0.6264
BAB          0.8996
PORTFOLIO    1.4497
Name: Sharpe Ratio, dtype: float64

The porfolio sharpe ratio is very high compared to the individual factors. This may look very appealing but our analysis suffers from look ahead bias. We are calculating the weights on the factor porfolio $\theta$ assuming that we have information on the future returns. However, in reality, this is not the case. An out of sample analysis may indicate different results. The portfolio sharpe ratio will still outperform all the individual factors due to benefits from correlation of factor returns, but probably not by such a huge margin!

# **1.4**
**Find the optimal portfolio weights for each of the $N_t$ = 2000 assets that are used to form each of the nine factors. That is, having obtained the optimal $\theta$ vector, please explain in words how one would obtain the optimal portfolio weights for each of the $N_t$ = 2000 assets that are used to form each of the nine factors.**

The asset weights $w_t(\theta)$ can be obtained from:

$$w_t(\theta) = w_{b,t} + \frac{F_{1,t}\theta_1 + F_{2,t}\theta_2 + \cdots + F_{K,t}\theta_K}{N_t}$$

### Notation

- $w_t(\theta)$: $N_t \times 1$ vector of portfolio weights at time $t$
- $w_{b,t}$: $N_t \times 1$ vector of benchmark portfolio weights at time $t$ (market weights can be considered as benchmark)
- $F_{k,t}$: $N_t \times 1$ long-short characteristic portfolio obtained by standardizing the $k$-th firm-specific factor at time $t$
- $\theta_k$: Scalar weight on the $k$-th factor in the factor portfolio
- $N_t$: Number of firms at time $t$ 
- $K$: Total number of characteristics

### Simplified equation in matrix form

Define $F_t$ as the $N_t$ x $K$ matrix whose $k^{th}$ column is $F_{k,t}$. Then, 

$$w_t(\theta) = w_{b,t} + \frac{F_t\theta}{N_t}$$

-$N_t$ = 2000  
-$K$ = 9  
-$w_{b,t}$ is the market portfolio weight  
-$F_{k,t}$ is the standardasied return on 9 on firm characteristics (e.g., SMB, HML, UMD, etc.);  

The market weights $w_{b,t}$ can be obtained easily from the market captilisation of assets. The only unkown is the K x 1 vector $\theta$ that we just estimated in 1.2. Therefore we can now dertermine the asset weights $w_t(\theta)$ if know the assets involved.

## **2.1 Volatility timing**
Use mean-variance optimization to combine  
-The original (without timing) factor, $f_{t+1}$  
-The volatility-managed version of this factor, $f^{\sigma}_{t+1}$

In [36]:
def get_volatility_managed_returns(factor_returns, vol_window=12, c=1):
    """
    Create volatility-managed factor(s) following Moreira & Muir (2017).
    
    f*_t+1 = (c / σ_t(f)^2) * f_t+1
    
    Parameters:
    -----------
    factor_returns : pd.Series or pd.DataFrame
        Time series of factor returns (Series) or multiple factors (DataFrame)
    vol_window : int
        Window for calculating realized volatility (default: 12 months)
    c : float
        Scaling constant (default: 1)
        
    Returns:
    --------
    DataFrame : Volatility-managed factor returns
    """
    realized_vol = factor_returns.rolling(window=vol_window).std()
    lagged_vol = realized_vol.shift(1)
    vol_managed = (c / lagged_vol**2) * factor_returns
    #rename columns for volatility-managed returns
    return vol_managed.add_suffix('_sigma').dropna()

#===================================================================================================

def get_rolling_theta(factor_returns, risk_aversion, window=120):

    n_assets = factor_returns.shape[1]
    n_periods = factor_returns.shape[0] 
    #array to store weights in for loop
    theta = np.full((n_periods, n_assets), np.nan)

    for i in range(window, n_periods):
        sample = factor_returns.iloc[i - window: i]
        theta[i] = get_theta(returns=sample, risk_aversion= risk_aversion)

    return pd.DataFrame(theta, index=factor_returns.index, columns=factor_returns.columns)

# construct two sets of factor returns: (1)original and (2)combined with volatility-managed factors        
vol_managed_factors = get_volatility_managed_returns(df_data)
df_original = df_data.loc[vol_managed_factors.index[0]:]
df_combined = pd.concat([df_original, vol_managed_factors], axis=1)

#get rolling weights on both sets of factor returns
theta_combined = get_rolling_theta(df_combined, risk_aversion=gamma, window=120).dropna()
theta_original = get_rolling_theta(df_original, risk_aversion=gamma, window=120).dropna()

print('weights on original factors:\n')
theta_original.round(4)

weights on original factors:



,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB
Dates,,,,,,,,,
1978-02-28,0.4938,2.3960,4.3598,2.2861,-5.7079,0.0356,4.5436,9.2767,1.4879
1978-03-31,0.5205,2.5096,4.4331,2.3018,-5.6214,0.1529,4.6096,9.0874,1.4298
1978-04-30,0.4834,2.5520,4.3602,2.2351,-5.8316,0.0901,4.5715,9.4746,1.5617
1978-05-31,0.5087,2.6009,4.4587,2.2846,-5.8748,0.1214,4.6490,9.4494,1.5277
1978-06-30,0.5132,2.5826,4.3348,2.3168,-5.9154,0.1233,4.5331,9.6687,1.5258
...,...,...,...,...,...,...,...,...,...
2020-08-31,2.1271,-0.7663,-2.9683,-1.0827,2.5367,-0.9220,2.3295,-0.1568,3.6431
2020-09-30,2.3617,-0.6993,-2.9839,-0.5599,3.3993,-0.6692,2.2305,-0.7842,3.2529
2020-10-31,2.2661,-0.6484,-2.9581,-0.7691,3.5542,-0.6922,2.3571,-1.1847,3.2827


In [37]:
print('weights on combined factors:\n')
theta_combined.round(4)

weights on combined factors:



,Market,SMB,HML,RMW,CMA,UMD,ROE,IA,BAB,Market_sigma,SMB_sigma,HML_sigma,RMW_sigma,CMA_sigma,UMD_sigma,ROE_sigma,IA_sigma,BAB_sigma
Dates,,,,,,,,,,,,,,,,,,
1978-02-28,3.7363,0.8032,5.4734,-0.9416,-9.9400,-0.6031,3.9723,16.7874,0.8826,-0.0052,0.0011,-0.0007,0.0011,0.0016,0.0009,0.0002,-0.0016,0.0001
1978-03-31,3.5816,1.2804,5.5564,-0.9000,-8.8528,-0.5644,4.3204,15.6885,0.9985,-0.0049,0.0008,-0.0007,0.0010,0.0014,0.0008,0.0002,-0.0015,-0.0000
1978-04-30,3.5871,1.2818,5.5584,-0.9819,-9.2346,-0.6031,4.3984,16.3541,1.0997,-0.0049,0.0008,-0.0007,0.0010,0.0014,0.0008,0.0001,-0.0015,-0.0000
1978-05-31,3.6541,1.1196,5.3496,-1.2314,-8.9794,-0.7220,4.2110,16.4642,1.0547,-0.0051,0.0009,-0.0008,0.0010,0.0013,0.0009,0.0001,-0.0015,0.0000
1978-06-30,3.6094,0.9755,5.1001,-1.2761,-8.8542,-0.9471,3.8823,16.4899,1.0642,-0.0051,0.0007,-0.0008,0.0010,0.0014,0.0009,0.0002,-0.0015,-0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,1.8621,0.5841,-5.5924,2.1133,2.7353,-4.0651,0.8964,-0.3491,4.8733,0.0004,-0.0003,0.0013,-0.0006,-0.0005,0.0019,0.0008,0.0003,0.0001
2020-09-30,2.3442,0.5902,-5.5726,2.5915,5.0349,-3.3444,1.1876,-2.9361,3.6114,0.0003,-0.0003,0.0013,-0.0006,-0.0007,0.0017,0.0007,0.0007,0.0002
2020-10-31,2.1946,0.6537,-5.5540,2.0322,5.4198,-3.2974,1.3773,-4.2538,3.6570,0.0003,-0.0003,0.0013,-0.0005,-0.0007,0.0016,0.0006,0.0008,0.0002


## **2.2**
Compare the Sharpe ratios of  
-the portfolio with just the original factor  
-the portfolio that includes the volatility-timed factor.  

In [42]:
def get_portfolio_returns(factor_returns, theta):
    """
    Calculate in-sample and out-of-sample portfolio returns given factor returns and weights.
    
    Parameters:
    -----------
    factor_returns : pd.DataFrame
        DataFrame of factor returns
    theta : pd.DataFrame
        DataFrame of portfolio weights (same columns as factor_returns)
        
    Returns:
    --------
    pd.DataFrame : Time series of portfolio returns with two columns: 'In-Sample' and 'Out-of-Sample'
    """
    
    In_sample_rets = (theta * factor_returns).sum(axis=1) 

    #calculate out of sample portfolio returns as: weights(t-1) * returns(t)
    Out_of_sample_rets = (theta.shift() * factor_returns).sum(axis=1)  
    
    #skip first row as they do not have any prior weights, so cannot be used for out-of-sample returns
    return pd.DataFrame({'In_Sample': In_sample_rets, 'Out_of_Sample': Out_of_sample_rets}).iloc[1:] 

rets_vol_timed = get_portfolio_returns(df_combined, theta_combined)
rets_vol_timed = rets_vol_timed.add_prefix('Vol_timed_')

rets_original = get_portfolio_returns(df_original, theta_original)
rets_original = rets_original.add_prefix('Original_')

df_results = pd.concat([rets_original, rets_vol_timed], axis=1)
sharpe_ratio(df_results).rename('Sharpe Ratio').round(2)

Original_In_Sample         1.23
Original_Out_of_Sample     1.14
Vol_timed_In_Sample        1.20
Vol_timed_Out_of_Sample    1.11
Name: Sharpe Ratio, dtype: float64

## **2.3 What do you conclude from your analysis above?**
### Expected results:
The in-sample sharpe ratios are higher than the corresponding out-of-sample sharpe ratios. This is as espected because all strategies have lower out of sample performance compared to their in-sample counterparts.

### Unexpected results:
Unlike Moreira & Muir's results, our volatility timed portfolio has lower sharpe ratio than the original factor portfolio. This could be because:  
1) The Moreira–Muir paper uses daily returns, which smooths volatility estimation and reduces noise. Using monthly data produces poor volatility estimates. This leads to noisy volatility-timed factor which in turn results in Sharpe deterioration.
2) Volatility managed factors have massive returns which leads to an extreme covariance matrix. $\Sigma$ becomes ill-conditioned and $\Sigma^{-1}$ becomes unstable. This leads to noisy weights vector - $\theta$
3) Moreira & Muir did not face the issue because they optimized nothing over multiple factors. They worked with individual factors one at a time, for example: Managed-MKT, Managed-SMB, Managed-HML etc. Our problem is much harder than theirs :)

### Possible solution:
The root of our sharpe ratio deterioration occurs due to an ill-conditioned covariance matrix $\Sigma$. This could perhaps be solved by using shrinking methods on covariance matrix like constant correlation or uncorrelated factors.

## **2.4 List the limitations of the strategy of timing factors conditional on their volatilities. Could one implement this volatility-timing policy in practice?**
### Limitations
1) This strategy requires extreme and unrealistic leverage. When past volatility is low, 1/$\sigma^2$ can be very large.  

2) Moreira & Muir conduct in-sample analysis which results in a look ahead bias in their results. Cederburg, O’Doherty, Wang, and Yan (2020) show that the out of sample sharpe ratios are lower than the unmanaged counterpart. This poor out-of-sample performance for volatility-managed portfolios stems primarily from structural instability in the underlying spanning regressions. 

3) Factors other than the market have very high transaction costs because they take relatively large positions in small-cap stocks that are expensive to trade. This leads to very high transaction costs for managed portfolios. Barroso and Detzel (2021) show that turnover of volatility-managed factors is up to 15 times higher. Accounting for transaction costs demolishes the superior performance.

4) Lastly, we already discussed the the limitaion of ill-conditioned covariance matrix when constructiong a portfolio with multiple factors.

### Can it be implemented?
**YES!** But not in the raw form used in academic studies. In practice, we must impose several constraints such as:  
-Leverage cap (e.g., max 3× or 5×)  
-Volatility target (e.g., 10% or 12%)  
-Smooth scaling over time  
-Use 1/σ instead of 1/σ²  

Institutional investors often implement volatility targeting, but in a highly constrained, smoothed form — not the raw 1/$\sigma^2$ scaling. The raw academic volatility-timed factor is NOT tradable without modifications.